In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
# Imports
from common import display_input_data, check_input_data_indices, train_model, prepare_dataset, evaluate_model, \
    run_input_optimizer, calculate_accuracy, calculate_error_rate, run_input_optimizer_2
import pandas as pd
import ipywidgets as widgets
import numpy as np
import itertools
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [50]:
# Parameters
# Chose variables to create the model: shape, cross-profile exit, rising entrance, shards, and mounding exit

input_cols = ['shape', 'profile_exit', 'rising_entrance', 'shards','exit_mounding']
result_col = 'result'
# Load the train and the validation datasets for blade class prediction
train_dataset_file = '../DATA/Train/blade_prediction_train.xlsx'
test_dataset_file = '../DATA/Validation/blade_prediction_validation.xlsx'

In [51]:
# check that all inputs correspond 
check_input_data_indices(train_dataset_file, test_dataset_file, input_cols)

#preparing the test dataset 
from sklearn.model_selection import train_test_split
x, y = prepare_dataset(train_dataset_file, input_cols, result_col)
 # Instantiate the model
logReg = LogisticRegression(solver='liblinear', penalty='l1', C= 1.0)
logReg.fit(x, y)


Datasets are compatible


LogisticRegression(penalty='l1', solver='liblinear')

In [52]:
# def train_model(train_dataset_file, input_cols, result_col):
#     """Create a logistic regression model on the provided dataset (including an expected result column) and fits/trains it on the result column."""
#     x, y = prepare_dataset(train_dataset_file, input_cols, result_col)
#     
#     # Instantiate the model
#     logReg = LogisticRegression(solver='liblinear', penalty='l1', C= 1.0)
#     logReg.fit(x, y)
# 
#   
# 
#     return logReg







In [53]:

#here we have the mean score and the standard deviation
#print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

from sklearn.metrics import make_scorer, precision_score, f1_score, recall_score, roc_auc_score, classification_report
from sklearn.model_selection import cross_validate
# Define custom scoring metrics
scoring = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1_score': make_scorer(f1_score),
    'AUC_scorer': make_scorer(roc_auc_score),
    'accuracy': make_scorer(accuracy_score)
}

scores = cross_validate(logReg, x, y, cv=10, scoring= scoring, return_train_score= True)
#printing the score for each of the cv 
scores

#here we have the mean score and the standard deviation
#print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
#here is printing the score using the cross-validation


{'fit_time': array([0.00462198, 0.00256681, 0.00258017, 0.00259519, 0.00208426,
        0.00269389, 0.00231528, 0.00226688, 0.00226998, 0.00228429]),
 'score_time': array([0.00850058, 0.00758696, 0.00767112, 0.00639653, 0.00637436,
        0.00640655, 0.0067606 , 0.00674725, 0.00676727, 0.0068891 ]),
 'test_precision': array([0.77777778, 0.79166667, 0.58333333, 0.58823529, 0.90909091,
        0.86956522, 0.83333333, 0.86956522, 0.84210526, 0.94736842]),
 'train_precision': array([0.88505747, 0.83333333, 0.85638298, 0.86111111, 0.81868132,
        0.78672986, 0.79047619, 0.82162162, 0.83977901, 0.8258427 ]),
 'test_recall': array([0.7 , 0.95, 0.35, 0.5 , 1.  , 1.  , 1.  , 1.  , 0.8 , 0.9 ]),
 'train_recall': array([0.85555556, 0.83333333, 0.89444444, 0.86111111, 0.82777778,
        0.92222222, 0.92222222, 0.84444444, 0.84444444, 0.81666667]),
 'test_f1_score': array([0.73684211, 0.86363636, 0.4375    , 0.54054054, 0.95238095,
        0.93023256, 0.90909091, 0.93023256, 0.82051282, 0.923

In [54]:
x_test, y_test = prepare_dataset(test_dataset_file, input_cols, result_col)
y_pred = logReg.predict(x_test)


#assuming y_true contains the true labels and y_pred contains the predicted labels
#Calculate precision

accuracy_score = accuracy_score(y_test, y_pred)
precision = precision_score(y_test,y_pred)

# Calculate recall
recall = recall_score(y_test, y_pred)

# Calculate F1 score
f1 = f1_score(y_test, y_pred)
#Calculate the ROC-AUC scorer
ROC_AUC = roc_auc_score(y_test, y_pred)

class_report = classification_report(y_test, y_pred)
print(class_report)
# Print or use the calculated precision, recall, and F1 score
print('accuracy score', accuracy_score)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)
print('ROC AUC:', ROC_AUC)

              precision    recall  f1-score   support

       False       0.81      0.65      0.72        20
        True       0.86      0.94      0.90        47

    accuracy                           0.85        67
   macro avg       0.84      0.79      0.81        67
weighted avg       0.85      0.85      0.85        67

accuracy score 0.8507462686567164
Precision: 0.8627450980392157
Recall: 0.9361702127659575
F1 Score: 0.8979591836734694
ROC AUC: 0.7930851063829787


In [55]:
def evaluate_overfitting(train_dataset_file, test_dataset_file, input_combination, result_col):

    model = train_model(
        train_dataset_file,
        list(input_combination),
        result_col
    )
    score_validation = evaluate_model(
        model,
        test_dataset_file,
        list(input_combination),
        result_col
    )
    score_train = evaluate_model(
        model,
        train_dataset_file,
        list(input_combination),
        result_col
    )

    accuracy_loss = score_train - score_validation
     # print(score)
    return model, score_validation, score_train, accuracy_loss




In [56]:
run_input_optimizer_2(train_dataset_file, test_dataset_file, input_cols, result_col)

Testing combinations of length 5


[]

In [57]:
#print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
evaluate_model(logReg, test_dataset_file, input_cols, result_col)

0.8507462686567164

In [58]:
evaluate_model(logReg, train_dataset_file, input_cols, result_col)

0.7642857142857142

In [59]:
param_grid = {

    'solver': ['liblinear'],
    'C': [100, 10, 1.0, 0.1, 0.01],
    'penalty': ['l2']
}

grid_search = GridSearchCV(estimator = LogisticRegression(), refit='accuracy',  param_grid= param_grid, scoring=['accuracy'], cv=10,)


grid_result = grid_search.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

NameError: name 'X_train' is not defined